# Sweep Comparison Notebook

このノートでは `rank-bandit-lab-sweep` コマンドや Python API を使って複数方策の結果を収集し、CTR とリグレットを比較します。

事前に `uv pip install matplotlib` を済ませた上で、CLI でログを生成しておくか、ノートブック内から直接実行してください。

In [ ]:
from pathlib import Path

from rank_bandit_lab import plot_learning_curve, plot_regret_curve, scenario_loader
from rank_bandit_lab.logging import load_log

OUTPUT_DIR = Path("sweep_logs")
OUTPUT_DIR.mkdir(exist_ok=True)
OUTPUT_DIR

In [ ]:
scenario_loader.list_scenarios()

## 1. CLI からログ生成

スイープを CLI 経由で実行する場合は以下のようにします。`uv` が入っていれば Python から `subprocess` で呼び出すこともできます。

In [ ]:
cmd = [
    "rank-bandit-lab-sweep",
    "--scenario",
    "video_streaming",
    "--run",
    "eps05:algo=epsilon,epsilon=0.05",
    "--run",
    "ts:algo=thompson,alpha_prior=1.0,beta_prior=1.0",
    "--run",
    "ucb07:algo=ucb,ucb_confidence=0.7",
    "--steps",
    "3000",
    "--slate-size",
    "4",
    "--output-dir",
    str(OUTPUT_DIR),
    "--summary-json",
    str(OUTPUT_DIR / "summary.json"),
]
print(" ".join(cmd))
# subprocess.run(cmd, check=True)  # 実際に実行する場合はコメントアウトを外してください

すでに `OUTPUT_DIR` にログがある場合は、そのまま `load_log` や `rank-bandit-lab-compare` で読み込み可能です。

In [ ]:
logs = sorted(OUTPUT_DIR.glob("*.json"))
logs

## 2. ログをロードして可視化

ここでは最初のログを読み込み、学習曲線とリグレット曲線を描きます。

In [ ]:
if logs:
    log, metadata = load_log(logs[0])
    metadata, log.summary()["ctr"]
else:
    print("No logs found. Run the sweep command above first.")

In [ ]:
if logs:
    plot_learning_curve(log)
    plot_regret_curve(log)

## 3. まとめ表の可視化

`rank-bandit-lab-compare` を呼び出すと CLI から比較表を得られますが、Notebook でも同様にできます。

In [ ]:
from rank_bandit_lab.compare import summaries_to_table, summarize

summaries = [summarize(str(path))[0] for path in logs]
print(summaries_to_table(summaries))